In [1]:
import json
import optuna
import subprocess
import time
from deploy_target import TargetDeployer
import requests
PROMETHEUS='http://prometheus.local'

In [2]:
target = TargetDeployer()
target.update_deployment_limits(120, 120)


In [3]:
def wait_for_experiment_state_change():
    while True:
        response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'max_over_time(k6_checks[5s])'}) 
        k = response.json()
        if len(k.get('data').get('result')) == 0:
            print('experiment ended')
            break;

In [4]:
def wait_for_failure():
    while True:
        response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'max_over_time(k6_http_req_failed[5s])'}) 
        k = response.json()
        if len(k.get('data').get('result')) != 0:
            response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'sum_over_time(k6_http_req_failed[1m])'})
            k = response.json()
            len(k.get('data').get('result')) > 1:
                print('run failed')
                return True
            else 
                print('run success')
                return False

In [5]:
def reset_experiment():
    r = requests.get('http://tester.local/reset')
    print(r.text)

In [6]:
def start_experiment():
    r = requests.get('http://tester.local/start')
    print(r.text)

In [7]:
def get_avg_latency():
    response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'avg_over_time(k6_custom_target_highmem_duration_avg[3m])'}) 
    print(response.json())
    k = response.json().get('data').get('result')
    print(k)
    if len(k) == 0:
        return None
    else:
        return k[0].get('value')[1]

In [8]:
def objective(trial):
    config = {
        "cpus" : trial.suggest_int("cpus", 100, 300, step=10),
        "memory" : trial.suggest_int("memory", 100, 300, step=10),
    }

    cpus = config.get('cpus')
    memory = config.get('memory')
    print("Running study with cpu: {} and memory: {}".format(cpus, memory))
    target.update_deployment_limits(cpus, memory)
    
    reset_experiment();
    wait_for_experiment_state_change()
    print("no experiment running")

    start_experiment()
    run_failed = wait_for_failure()
    if run_failed:
        print("experiment failed")
        raise optuna.TrialPruned()
    else: 
        latency = get_avg_latency();
        print("latency: {}".format(latency))
    
    reset_experiment();
    wait_for_experiment_state_change()
    print("experiment stopped")

#     if trial.should_prune():
#         raise optuna.TrialPruned()

    return float(latency)

In [11]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)

[I 2022-07-21 10:04:06,350] A new study created in memory with name: no-name-b2473ff3-3647-4a95-9662-0417a9decd5e


Reset done
experiment ended
no experiment running
Started


[I 2022-07-21 10:05:06,688] Trial 0 pruned. 


{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'error': 'EOF', 'error_code': '1000', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_04-34-21', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658378106.696, '0.2']}, {'metric': {'error': 'dial: connection refused', 'error_code': '1212', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_04-33-25', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658378106.696, '27.25311124777207']}, {'metric': {'error': 'dial: connection refused', 'error_code': '1212', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_04-34-21', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658378106.696, '125.94553322521715']}, {'metric': {'erro

[I 2022-07-21 10:05:31,982] Trial 1 pruned. 


...

experiment ended
experiment stopped
Reset done
experiment ended
no experiment running
Started
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'expected_response': 'true', 'method': 'GET', 'proto': 'HTTP/1.1', 'run': '2022-07-21_05-31-06', 'scenario': 'target_highmem', 'status': '200', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658381517.045, '0']}, {'metric': {'expected_response': 'true', 'method': 'GET', 'proto': 'HTTP/1.1', 'run': '2022-07-21_05-31-47', 'scenario': 'target_highmem', 'status': '200', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658381517.045, '0']}]}}
['200', '200']
[]
run success
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'run': '2022-07-21_05-26-04', 'scenario': 'target_highmem', 'tag': 'highmem_duration'}, 'value': [1658381522.115, '56.607429379372284']}, {'metric': {'run': '2022-07-21_05-29-45', 'scenario': 'target_hi

[I 2022-07-21 11:02:12,235] Trial 28 finished with value: 56.607429379372284 and parameters: {'cpus': 290, 'memory': 280}. Best is trial 14 with value: 24.358422481949408.


experiment ended
experiment stopped
Reset done
experiment ended
no experiment running
Started
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'expected_response': 'true', 'method': 'GET', 'proto': 'HTTP/1.1', 'run': '2022-07-21_05-31-47', 'scenario': 'target_highmem', 'status': '200', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658381557.817, '0']}, {'metric': {'expected_response': 'true', 'method': 'GET', 'proto': 'HTTP/1.1', 'run': '2022-07-21_05-32-27', 'scenario': 'target_highmem', 'status': '200', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658381557.817, '0']}]}}
['200', '200']
[]
run success
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'run': '2022-07-21_05-29-45', 'scenario': 'target_highmem', 'tag': 'highmem_duration'}, 'value': [1658381562.838, '92.19655642963048']}, {'metric': {'run': '2022-07-21_05-30-26', 'scenario': 'target_hig

[I 2022-07-21 11:02:52,873] Trial 29 finished with value: 92.19655642963048 and parameters: {'cpus': 260, 'memory': 210}. Best is trial 14 with value: 24.358422481949408.


experiment ended
experiment stopped


In [12]:
best_params = study.best_params
print(best_params)

{'cpus': 250, 'memory': 270}


In [15]:
def get_latency(trial):
    return trial.values[0]
fig = optuna.visualization.plot_contour(study, params=["cpus", "memory"], target=get_latency, target_name="Latency")
fig.show()
# def get_throughput(trial):
#     return trial.values[1]
# fig1 = optuna.visualization.plot_contour(study, params=["cpus", "memory"], target=get_throughput, target_name="Throughput")
# fig1.show()